<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/scripts/Finetuning_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()

In [2]:
!pip install transformers datasets accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.3 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()

Saving test_data.csv to test_data.csv
Saving train_data.csv to train_data.csv
Saving validation_data.csv to validation_data.csv


In [4]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch

# Load your CSVs
data_files = {
    "train": "train_data.csv",
    "validation": "validation_data.csv",
    "test": "test_data.csv",
}
dataset = load_dataset("csv", data_files=data_files)

# Inspect
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
})


In [5]:
model_name = "google/medgemma-4b-pt"  # or the version you already downloaded

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True
)


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Gemma3ForSequenceClassification were not initialized from the model checkpoint at google/medgemma-4b-pt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir="./medgemma-eczema-binary-classification",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)



In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

model_name = "google/medgemma-4b-pt"

# 1️⃣ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2️⃣ Load model in 4-bit for memory efficiency
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # binary classification
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# 3️⃣ Add LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # typical attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 4️⃣ Define training arguments
training_args = TrainingArguments(
    output_dir="./medgemma-eczema-binary",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)

# 5️⃣ Train as before
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer
)




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Gemma3ForSequenceClassification were not initialized from the model checkpoint at google/medgemma-4b-pt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,452,224 || all params: 4,306,536,816 || trainable%: 0.1498


/tmp/ipython-input-3951112201.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan


TrainOutput(global_step=2250, training_loss=0.0, metrics={'train_runtime': 6145.6861, 'train_samples_per_second': 1.464, 'train_steps_per_second': 0.366, 'total_flos': 2.5094470422528e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [13]:
trainer.evaluate()


{'eval_loss': nan,
 'eval_runtime': 498.8915,
 'eval_samples_per_second': 4.009,
 'eval_steps_per_second': 1.002,
 'epoch': 3.0}

In [14]:
import pandas as pd
df = pd.read_csv("train_data.csv")
print(df.head())
print(df["text"].nunique(), "/", len(df))
print(df["label"].value_counts())


  disease                                               text  label
0  eczema  Blistering or weeping lesions in acute flares ...      1
1  eczema  Faintly discolored patches that are darker or ...      1
2  eczema  Intensely itchy, dry and scaly patches of skin...      1
3  eczema  The patient has intensely itchy, dry and scaly...      1
4     NaN                       My head aches when I wake up      0
2009 / 3000
label
1    2000
0    1000
Name: count, dtype: int64
